In [ ]:
# Initial notebook setup -- RUN ME FIRST!
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import rcParams
rcParams.update({'font.size': 14})
rcParams['figure.figsize'] = (10, 5)


# Numerical solutions for stellar structure

We have spent the last several weeks deriving a set of equations that can be solved to find the pressure, temperature, density, luminosity, etc, as functions of radius in a star. On problem set 3, you solved these equations by making the unrealistic assumption that density is constant with radius. There are a couple other cases that we can solve analytically, but the general problem of stellar structure is more easy to study by numerical calculation.

As a reminder, here are the equations of stellar structure as presented in Maoz Section 3.5:

$$
\begin{align}
\frac{dP(r)}{dr} &= - \frac{G M(r) \rho(r)}{r^2} &\quad \mathrm{(hydrostatic~equilibrium)} \\
\frac{dM(r)}{dr} &= 4 \pi r^2 \rho(r) &\quad \mathrm{(mass~conservation)} \\
\frac{dT(r)}{dr} &= - \frac{3 L(r) \kappa(r) \rho(r)}{4 \pi r^2 4 a c T(r)^3} &\quad \mathrm{(radiative~transfer)} \\
\frac{dL(r)}{dr} &= 4 \pi r^2 \rho(r) \epsilon(r) &\quad \mathrm{(energy~conservation)} \\
P &= P(\rho, T, \mathrm{composition}) &\quad \mathrm{(equation~of~state)} \\
\kappa &= \kappa(\rho, T, \mathrm{composition}) &\quad \mathrm{(opacity)} \\
\epsilon &= \epsilon(\rho, T, \mathrm{composition}) &\quad \mathrm{(power~density)}
\end{align}
$$

The differential equations are coupled and non-linear, which makes it impossible to write a general analytic solution. However, we can find an approximate solution if we assume that all of these functions of radius are constant across the small $dr$ of each thin spherical shell. This should sound familiar, because it is exactly the description that we used to derive the differential equations above.

Take the equation of mass conservation as an example. Suppose that we know $M(r)$ and $\rho(r)$ for some particular radius $r$. Then we can make a small step out by interval $\Delta r$, as follows:

$$
\begin{align}
\frac{dM}{dr} &= 4 \pi r^2 \rho(r) \\
\frac{M(r + \Delta r) - M(r)}{\Delta r} &= 4 \pi r^2 \rho(r) \\
M(r + \Delta r) &= M(r) + 4 \pi r^2 \rho(r) \Delta r
\end{align}
$$

Using this similar strategy with the other equations, we can "build" our star from the inside out. This technique for solving differential equations is known as [Euler's method](https://en.wikipedia.org/wiki/Euler_method).


In [ ]:
# Define some physical constants. 
# It will be important to keep our units straight, or else we will get crazy results.
G = 6.7e-8       # erg cm g^{-2}             # Newton's constant
c = 3.0e10       # cm s^{-1}                 # speed of light
k = 1.4e-16      # erg cm^{-2} s^{-1} K^{-4} # Boltzmann's constant
a = 7.6e-15      # erg cm^{-3} K^{-4}        # radiation constant
sigmaT = 6.7e-25 # cm^2                      # Thomson cross-section
mH = 1.7e-24     # g                         # mass of Hydrogen


## Differential equations

Now, let's write one function for each of the four differential equations. For these functions, just calculate the right-hand side of the appropriate equation. Later on, we will multiply by $\Delta r$ and use that value to increment the value of $P$, $M$, $T$, or $L$ from $r$ to $r + \Delta r$.

I have written out one example (mass conservation) for you. Remember that you have to run each code block cell to define the function writen in it.


In [ ]:
def M_deriv(r, rho):
    """
    Calculate dM/dr using equation of mass conservation.
    
    Inputs
    ------
    r : float
        Radius from center of the star, in cm
    rho : float
        Matter density at radius r, in g cm^{-3}
    
    Returns
    -------
    dMdr : float
        Derivative of mass function w.r.t. radius, in g cm^{-1}
    
    """
    
    dMdr = 4.0 * np.pi * r**2 * rho # Note: np.pi = 3.14159265359...
    return dMdr


In [ ]:
def P_deriv(r, M, rho):
    """
    Calculate dP/dr using equation of hydrostatic equilibrium.
    
    Inputs
    ------
    r : float
        Radius from center of the star, in cm
    M : float
        Total mass enclosed by a sphere of radius r, in g
    rho : float
        Matter density at radius r, in g cm^{-3}
        
    Returns
    -------
    dPdr : float
        Derivative of pressure w.r.t. radius, in erg cm^{-4}
    
    """
    
    # ADD CODE HERE TO CALCULATE dPdr
    return dPdr


In [ ]:
def T_deriv_rad(r, L, kappa, rho, T):
    """
    Calculate dT/dr using equation of radiative transfer.
    
    Inputs
    ------
    r : float
        Radius from center of the star, in cm
    L : float
        Luminosity at radius r, in erg s^{-1}
    kappa : float
        Opacity function at radius r, in cm^2 g^{-1}
    rho : float
        Matter density at radius r, in g cm^{-3}
    T : float
        Temperature at radius r, in K
        
    Returns
    -------
    dTdr : float
        Derivative of temperature w.r.t. radius, in K cm^{-1}
    
    """
    
    # ADD CODE HERE TO CALCULATE dTdr
    return dTdr


In [ ]:
def L_deriv(r, rho, epsilon):
    """
    Calculate dL/dr using equation of energy conservation.
    
    Inputs
    ------
    r : float
        Radius from center of the star, in cm
    rho : float
        Matter density at radius r, in g cm^{-3}
    epsilon : float
        Power density at radius r, in erg s^{-1} g^{-1}
        
    Returns
    -------
    dLdr : float
        Derivative of luminosity w.r.t. radius, in erg s^{-1} cm^{-1}
        
    """
    
    # ADD CODE HERE TO CALCULATE dLdr
    return dLdr


## Equation of state

We also need an equation of state, which relates pressure ($P$) and density ($\rho$). For this exercise, let's assume the **ideal gas law**:

$$
\begin{equation}
P = \frac{\rho}{\bar{m}} k T
\end{equation}
$$

However, we will be using the equation of hydrostatic equilibrium to iteratively solve for pressure, so let's invert this equation to obtain an expression for $\rho$:

$$
\begin{equation}
\rho = \frac{\bar{m}}{k T} P
\end{equation}
$$

Our function will include $\bar{m}$, the average particle mass, as an input argument so that we can use this same function for different choices of composition.


In [ ]:
def calc_rho(P, T, mbar):
    """
    Calculates density using ideal gas equation of state.
    
    Inputs
    ------
    P : float
        Pressure, in erg cm^{-3}
    T : float
        Temperature, in K
    mbar : float
        Average particle mass, in g
        
    Returns
    -------
    rho : float
        Mass density, in g cm^{-3}
    
    """
    
    # ADD CODE HERE TO CALCULATE rho
    return rho


## Opacity

We will assume that the opacity is entirely due to Thomson scattering. For that case, $\kappa$ depends only on composition, specifically the Hydrogen fraction $X$.

$$
\begin{equation}
\kappa = \frac{\sigma_T}{2 m_H} (1 + X)
\end{equation}
$$

Under what conditions is Thomson scattering going to be a bad assumption?

In [ ]:
def calc_kappa(X):
    """
    Calculates opacity from Thomson scattering.
    
    Inputs
    ------
    X : float
        Hydrogen mass fraction
        
    Returns
    -------
    kappa : float
        Opacity, in cm^2 g^{-1}
    
    """
    
    # ADD CODE HERE TO CALCULATE kappa
    return kappa


## Power density

Our star is on the main sequence, and we will assume that all of its nuclear energy production happens via the $p-p$ chain. The power density, $\epsilon$, is a function of $\rho$ and $T$. We also need to specify a bunch of additional constants. 

This function is provided for you.


In [ ]:
# Constants for p-p chain
Q = 26.2 * 1e6 * 1.6e-12 # energy released by set of reactions; 26.2 MeV converted to erg
S0 = 4.0e-46 * 1e3 * 1.6e-12 # nuclear cross-section constant; 4e-46 cm^2 keV converted to cm^2 erg
EG = 500.0 * 1e3 * 1.6e-12 # Gamow energy; 500 keV converted to erg

def calc_epsilon(rho, T, X):
    """
    Calculates power density generated via p-p chain.
    
    Inputs
    ------
    rho : float
        Matter density, in g cm^{-3}
    T : float
        Temperature, in K
    X : float
        Hydrogen mass fraction
        
    Returns
    -------
    epsilon : float
        Power density, in erg s^{-1} g^{-1}
    
    """
    
    mu = mH / 2 # Reduced mass for two protons
    epsilon = ((2.0**(5./3.) * np.sqrt(2) / np.sqrt(3)) * ((rho * X**2) / (mH**2 * np.sqrt(mu))) * 
               (Q * S0) * (EG**(1./6.) / (k * T)**(2./3.)) * np.exp(-3.0 * (EG / (4.0 * k * T))**(1./3.)))
    return epsilon
    

## Added complication: convection

To save time, we skipped over convection, which is discussed in Maoz Section 3.12. However, this effect is important to include if we want to get realistic-looking stars.

Convection is an alternate means to transport energy, other than radiative transport. In radiative transport, the gas stays put and energy is carried by light. In convection, gas moves between different radii to carry energy. But convection is only effective in certain situations. For an example where convection is *not effective*, think of a still pond on a sunny day, which can develop a thin layer of warm water near the surface while remaining cold below.

Mathematically, we can write the following inequality that tells us when convection will occur:

$$
\begin{equation}
\left| \frac{dT}{dr} \right| \gt \frac{\gamma - 1}{\gamma} \frac{T}{P} \left| \frac{dP}{dr} \right|
\end{equation}
$$

The symbol $\gamma$ in this expression is the adiabatic index of the gas, which has to do with the gas particle degrees-of-freedom. We will use $\gamma = 5/3$, which is the value for a monatomic gas (like ionized Hydrogen).

If the magnitude of the temperature gradient is larger than the term on the right hand side of the inequality, then convection will occur. When it does occur, it is a very efficient method of energy transport so it will quickly reduce the magnitude of the temperature gradient. So for our model, we will handle convection in the following way:

1. Calculate $dT/dr$ from radiative transfer, using the function you wrote above.
2. Compare this $dT/dr$ to the right hand side of the convection inequality.
3. If the magnitude of $dT/dr$ from radiative transport is large enough for convection to occur, then it gets capped at the limit where convection turns on.

Using this algorithm, we can write a function for $dT/dr$ that incorporates both radiative transfer and convection. The combined function is provided for you below, but it makes use of the `T_deriv_rad` function that you wrote above.


In [ ]:
# Another constant.
gamma = 5.0 / 3.0 # Adiabatic index for monatomic gas

def T_deriv(r, L, kappa, rho, T, P, dPdr):
    """
    Calculate dT/dr from radiative transfer *or* convection.
    
    Inputs
    ------
    r : float
        Radius from center of the star, in cm
    L : float
        Luminosity at radius r, in erg s^{-1}
    kappa : float
        Opacity function at radius r, in cm^2 g^{-1}
    rho : float
        Matter density at radius r, in g cm^{-3}
    T : float
        Temperature at radius r, in K
    P : float
        Pressure at radius r, in erg cm^{-3}
    dPdr : float
        Derivative of pressure w.r.t. radius, in erg cm^{-4}
        
    Returns
    -------
    dTdr : float
        Derivative of temperature w.r.t. radius, in K cm^{-1}
    
    """
    
    # Calculate dT/dr for radiative transport
    dTdr_rad = T_deriv_rad(r, L, kappa, rho, T)
    # Calculate threshold for convection
    dTdr_conv = ((gamma - 1.0) / gamma) * (T / P) * dPdr
    # Decide whether convection or radiative transport is the dominant process
    if np.abs(dTdr_rad) > np.abs(dTdr_conv):
        # Convection is happening
        dTdr = dTdr_conv
    else:
        # No convection
        dTdr = dTdr_rad
    # Done
    return dTdr


## Iterative solution for stellar structure

Now we have all the necessary pieces to calculate stellar structure. The final function you write will choose some range of radius values with small steps between them. Then it will start at the inside of the star and works its way out. Remember how our step-wise method works -- use the functions you wrote to calculate derivatives, multiply by $\Delta r$ to get the step size, then add that step onto the value from the previous shell.

We need four boundary conditions for the differential equations, so we will use the following:

* $M(0) = 0$
* $L(0) = 0$
* $T(0) = T_0$
* $P(0) = P_0$

Since we don't know the radius of the star until we calculate it, we will try to specify a range of radius that is larger than we need but then cut it off once the pressure falls to zero.

Both the equations of hydrostatic equilibrium and radiative transfer have factors of $r$ in the denominator. If our very first step is `r[0] = 0`, then we will run into a divide-by-zero error. To avoid this, we will shift the array of `r` values up by half a step. For sufficiently small steps, this shouldn't make any difference in the final result.


In [ ]:
def make_star(T0, P0, X, Y):
    """
    Solve equations of stellar structure for a main sequence star with opacity due to Thomson scattering.
    
    Inputs
    ------
    T0 : float
        Temperature at the center of the star, in K
    P0 : float
        Pressure at the center of the star, in erg cm^{-3}
    X : float
        Hydrogen mass fraction 
    Y : float
        Helium mass fraction
        (NOTE: metals fraction Z = 1 - X - Y)
        
    Returns
    -------
    r : array
        Radius values, in cm
    M : array
        Mass function, in g
    P : array
        Pressure function, in erg cm^{-3}
    T : array
        Temperature function, in K
    L : array
        Luminosity function, in erg s^{-1}
    rho : array
        Mass density function, in g cm^{-3}
    kappa : array
        Opacity function, in cm^2 g^{-1}
    epsilon : array
        Power density function, in erg s^{-1} g^{-1}
    
    """
    
    # Calculate average particle mass based on composition
    mbar = mH * 2.0 / (1.0 + 3.0 * X + 0.5 * Y)
    
    # Use a large range of radius, so we capture the whole star.
    # If you try some extreme cases, this might not be enough. In that case, extend it further!
    rmax = 1.0e12 # cm
    nstep = 100000 # number of steps
    r = np.linspace(0, rmax, nstep) # array of equally spaced r values
    dr = r[1] - r[0] # Delta_r
    r = r + dr / 2.0 # Shift array up by half a step to avoid dividing by zero.
    
    # The seven functions we are going to calculate should all have the
    # same number of entries as r.
    # Initialize these arrays will all zeros to start.
    M = np.zeros(r.shape)
    P = np.zeros(r.shape)
    T = np.zeros(r.shape)
    L = np.zeros(r.shape)
    rho = np.zeros(r.shape)
    kappa = np.zeros(r.shape)
    epsilon = np.zeros(r.shape)
    
    # Set up initial conditions.
    # Calculate the first entry for all of our functions.
    M[0] = 0.0
    P[0] = P0
    T[0] = T0
    L[0] = 0.0
    rho[0] = calc_rho(P[0], T[0], mbar)
    kappa[0] = calc_kappa(X)
    epsilon[0] = calc_epsilon(rho[0], T[0], X)
    
    # Now we are going to start in the center of the star at r[0]
    # and work our way out, step by step.
    # Variable 
    for i in range(1, nstep):
        # **ADD CODE HERE**
        # Add one step to M, P, T, and L using the differential equations.
        # Then update rho, kappa, and epsilon
        
        # After we have updated all seven functions, we will check to see
        # if pressure, temperature, or density has dropped to zero, which 
        # indicates that we reached the edge of the star.
        if (P[i] <= 0.0) or (T[i] <= 0.0) or (rho[i] <= 0.0):
            break # Stops the for loop
    
    # Calculate and print r_star, M_star, and L_star
    print("r_star = {}".format(r[i]))
    print("M_star = {}".format(M[i]))
    print("L_star = {}".format(L[i]))
    
    # Cut off unused entries from these arrays.
    r = r[0:i+1]
    M = M[0:i+1]
    P = P[0:i+1]
    T = T[0:i+1]
    L = L[0:i+1]
    rho = rho[0:i+1]
    kappa = kappa[0:i+1]
    epsilon = epsilon[0:i+1]
    
    # Return results
    return (r, M, P, T, L, rho, kappa, epsilon)


## Test your code

Try running the cell below, which specifies a star with central pressure $P_0 = 10^{18} \; \mathrm{erg} \, \mathrm{cm}^{-3}$ (about $10^{12}$ atmospheres), central temperature $T_0 = 1.5 \times 10^7$ K, and composition that is 70% Hydrogen, 28% Helium, and 2% metals.

Then, try adjusting the parameters to see what you find!

In [ ]:
# Calculate stellar structure for specified parameters.
P0 = 1.0e18 # erg cm^{-3}
T0 = 1.5e7 # K
X = 0.7
Y = 0.28
(r, M, P, T, L, rho, kappa, epsilon) = make_star(T0, P0, X, Y)

# Set up a big area for plots of all seven functions.
rcParams['figure.figsize'] = (16, 20)

# Plot mass in units of solar mass.
rsun = 7.0e10 # cm
Msun = 2.0e33 # g
plt.subplot(4, 2, 1) # row 1 column 1 of a 4x2 plot grid
plt.plot(r / rsun, M / Msun)
plt.xlabel('r / r_Sun')
plt.ylabel('M / M_Sun')

# Plot pressure relative to central pressure.
plt.subplot(4, 2, 2) # row 1 column 2 of a 4x2 plot grid
plt.plot(r / rsun, P / P0)
plt.xlabel('r / r_Sun')
plt.ylabel('P / P_0')

# Plot temperature relative to central temperature.
plt.subplot(4, 2, 3) # row 2 column 1 of a 4x2 plot grid
plt.plot(r / rsun, T / T0)
plt.xlabel('r / r_Sun')
plt.ylabel('T / T_0')

# Plot luminosity in units of solar luminosity.
Lsun = 3.8e33 # erg s^{-1}
plt.subplot(4, 2, 4) # row 2 column 2 of a 4x2 plot grid
plt.plot(r / rsun, L / Lsun)
plt.xlabel('r / r_Sun')
plt.ylabel('L / L_Sun')

# Plot density in g cm^{-3}
plt.subplot(4, 2, 5) # row 3 column 1 of a 4x2 plot grid
plt.plot(r / rsun, rho)
plt.xlabel('r / r_Sun')
plt.ylabel('rho [g cm^-3]')

# Plot opacity in cm^2 gm^{-1}
plt.subplot(4, 2, 6) # row 3 column 2 of a 4x2 plot grid
plt.plot(r / rsun, kappa)
plt.xlabel('r / r_Sun')
plt.ylabel('kappa [cm^2 g^-1]')

# Plot power density in erg s^{-1} g^{-1}
plt.subplot(4, 2, 7) # row 4 column 1 of a 4x2 plot grid
plt.plot(r / rsun, epsilon)
plt.xlabel('r / r_Sun')
plt.ylabel('epsilon [erg s^-1 g^-1]')


## Radiation pressure

For the equation of state, we assumed the ideal gas law. But we know that there is another contribution to pressure from radiation:

$$
P_\mathrm{rad} = \frac{1}{3} a T^4
$$

Since we know the temperature function, we can calculate the radiation pressure everywhere in the star. Compare the radiation pressure to the gas pressure that you already calculated. Is it a good approximation to ignore radiation pressure? (the answer might depend on what parameters you used to build your star)